![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FValidate&file=Vertex+AI+Agent+Builder+Check+Grounding+API.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Validate/Vertex%20AI%20Agent%20Builder%20Check%20Grounding%20API.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FValidate%2FVertex%2520AI%2520Agent%2520Builder%2520Check%2520Grounding%2520API.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Validate/Vertex%20AI%20Agent%20Builder%20Check%20Grounding%20API.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Validate/Vertex%20AI%20Agent%20Builder%20Check%20Grounding%20API.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Vertex AI Agent Builder Check Grounding API

Large Language Models (LLMs) are powerful tools for generating human-like text, but they can sometimes generate inaccurate information. To ensure your LLM provides grounded and factual responses, you need to provide it with relevant context.  Retrieval augemented generation (RAG) help [find the relevant context](../Retrieval/readme.md) and [Ranking](../Ranking/readme.md) can help filter and sort the retrieved context. But how can you be sure the LLM is actually using that context effectively?

That's where the Vertex AI Agent Builder [Check Grounding API](https://cloud.google.com/generative-ai-app-builder/docs/check-grounding) comes in. This API helps you analyze how well an LLM's response is grounded in the context you provided.

Here's how it works:

- Input: You provide the Check Grounding API with the LLM's response and the context chunks you included in the prompt.
- Analysis: The API breaks down the LLM's response and maps phrases to supporting evidence within the context.
- Output: The API returns a detailed report, including:
    - Citations: Links between specific phrases in the response and the supporting context.
    - Support Score: An overall score (0 to 1) indicating how well the response is supported by the context.

The Check Grounding API enables you to build more reliable and trustworthy LLM applications. You can use the support scores and citations to:
- Set thresholds for response quality: Ensure that only factually grounded responses are provided to users.
- Trigger advanced prompting techniques: If a response is poorly grounded, automatically retrieve additional context or reword the question to improve accuracy.
- Gain insights into LLM behavior: Understand how the LLM utilizes context and identify areas for improvement in your prompting strategy.

**References:**

- [Discoveryengine Python Grounded Generation Service Client](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.grounded_generation_service)

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.66.0'),
    ('google.cloud.discoveryengine', 'google-cloud-discoveryengine', '0.12.2')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable discoveryengine.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

inputs:

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [9]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'validate-check-grounding'

packages:

In [10]:
import os, shutil, json

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from IPython.display import Markdown

import google.cloud.discoveryengine_v1 as discoveryengine
from google.cloud import aiplatform
import vertexai.generative_models # for Gemini Models
import vertexai.language_models # for text embedding models

In [11]:
aiplatform.__version__

'1.69.0'

In [12]:
discoveryengine.__version__

'0.12.3'

clients:

In [13]:
# Vertex AI
vertexai.init(project = PROJECT_ID, location = REGION)

# Vertex AI Agent Builder APIs
check_grounding_client = discoveryengine.GroundedGenerationServiceClient()

---
## Text & Embeddings For Examples

This repository contains a [section for document processing (chunking)](../Chunking/readme.md) that includes an [example of processing a PDF with the Document AI Layout Parser](../Chunking/Process%20Documents%20-%20Document%20AI%20Layout%20Parser.ipynb).  The chunks of text from that workflow are stored with this repository and loaded by another companion workflow that augments the chunks with text embeddings: [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb).

The following code will load the version of the chunks that includes text embeddings and prepare it for a local example of retrival augmented generation.

### Get The Documents

If you are working from a clone of this notebooks [repository](https://github.com/statmike/vertex-ai-mlops) then the documents are already present. The following cell checks for the documents folder and if it is missing gets it (`git clone`):

In [14]:
local_dir = '../Embeddings/files/embeddings-api'

In [15]:
if not os.path.exists(local_dir):
    print('Retrieving documents...')
    parent_dir = os.path.dirname(local_dir)
    temp_dir = os.path.join(parent_dir, 'temp')
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    !git clone https://www.github.com/statmike/vertex-ai-mlops {temp_dir}/vertex-ai-mlops
    shutil.copytree(f'{temp_dir}/vertex-ai-mlops/Applied GenAI/Embeddings/files/embeddings-api', local_dir)
    shutil.rmtree(temp_dir)
    print(f'Documents are now in folder `{local_dir}`')
else:
    print(f'Documents Found in folder `{local_dir}`')             

Documents Found in folder `../Embeddings/files/embeddings-api`


### Load The Chunks

In [16]:
with open(local_dir+'/chunk-embeddings.jsonl', 'r') as f:
    chunks = [json.loads(line) for line in f]

### Review A Chunk

In [17]:
chunks[0].keys()

dict_keys(['instance', 'predictions', 'status'])

In [18]:
chunks[0]['instance']['chunk_id']

'c2'

In [19]:
print(chunks[0]['instance']['content'])

# OFFICIAL BASEBALL RULES

## Official Baseball Rules 2023 Edition

### JOINT COMPETITION COMMITTEE

|-|-|-|
| Bill DeWitt | Whit Merrifield | Austin Slater |
| Jack Flaherty | Bill Miller | John Stanton, Chair |
| Tyler Glasnow | Dick Monfort | Tom Werner |
| Greg Johnson | Mark Shapiro |  |

Committee Secretary Paul V. Mifsud, Jr. Copyright © 2023 by the Office of the Commissioner of Baseball


In [20]:
chunks[0]['predictions'][0]['embeddings']['values'][0:10]

[0.008681542240083218,
 0.06999468058347702,
 0.003673204220831394,
 0.019888797774910927,
 0.016285404562950134,
 0.035664502531290054,
 0.06200747936964035,
 0.05597030743956566,
 0.0034793149679899216,
 -0.024485772475600243]

### Prepare Chunk Structure

Make a dictionary for each lookup of chunk content by chunk id:

In [21]:
content_chunks = {}
for chunk in chunks:
    content_chunks[chunk['instance']['chunk_id']] = chunk['instance']['content']

In [22]:
content_chunks['c1']

'# OFFICIAL BASEBALL RULES\n\n2023 Edition TM TM'

---
## Simple Retrieval Augmented Generation (RAG)

Embeddings can be used with math to measure similarity.  For deeper details into this checkout the companion workflow here: [The Math of Similarity](../Embeddings/The%20Math%20of%20Similarity.ipynb).  Retrieval systems handle the storage and math of similarity as a service.  For an overview of Google Cloud based solutions for retrieval check out [this companion series](../Retrieval/readme.md).

The content below motivates retrieval with the embeddings that accompany the text chunks using a local vector database with brute force matching using Numpy!

### Vector DB With Numpy

In [23]:
vector_db = [
    [
        chunk['instance']['chunk_id'],
        chunk['predictions'][0]['embeddings']['values'],
    ]
    for chunk in chunks
]
vector_index = np.array([row[1] for row in vector_db])

### Models: Embeddings, Generation

Connect to models for text embeddings and text generation:

In [24]:
embedder = vertexai.language_models.TextEmbeddingModel.from_pretrained('text-embedding-004')
llm = vertexai.generative_models.GenerativeModel("gemini-1.5-flash-001")

Define a question that is the start of our prompt to the LLM:

In [25]:
question = "What are the dimensions of a base?"

Get an ungrounded response to the question with the LLM:

In [26]:
print(llm.generate_content(question).text)

The term "base" is very general, and its dimensions will depend on the context.  To answer your question accurately, I need more information. 

Please tell me:

* **What kind of base are you talking about?**  Is it a base of a:
    * **Geometric shape?** (like a triangle, rectangle, pyramid, etc.)
    * **Object?** (like a table, a house, a building, etc.)
    * **Mathematical concept?** (like a number system, a logarithm, etc.)
* **What specific information are you looking for?**  Are you interested in:
    * **Length and width?**
    * **Area?**
    * **Volume?** 
    * **Other specific dimensions?**

Once you provide more context, I can help you determine the dimensions of the base you're interested in. 



Get an embedding for the question to use in retrieval:

In [27]:
question_embedding = embedder.get_embeddings([question])[0].values
question_embedding[0:10]

[-0.026682045310735703,
 0.011593513190746307,
 0.028523651883006096,
 -0.0017065361607819796,
 0.01946176588535309,
 0.0031198114156723022,
 0.07915323227643967,
 -0.005078596994280815,
 -0.006295712199062109,
 0.04943541809916496]

### Retrieval: Matching With Numpy

Use dot product to calculate similarity and find matches for a query embedding.  Why dot product?  Check out the companion workflow: [The Math of Similarity](../Embeddings/The%20Math%20of%20Similarity.ipynb)

> **NOTE:**  This will calculate the similarity for all embeddings vectors stored in the local vector db which is just a Numpy array here.  This is very fast because there are <200 embeddings vectors.  As this scales it would be better to consider a solution that searches a subset of embeddings.  More details on retrieval solutions can be found in [Retrieval](../Retrieval/readme.md).

In [33]:
similarity = np.dot(question_embedding, vector_index.T)
#matches = np.argsort(similarity)[::-1][:5].tolist()
matches = np.argsort(similarity)[-5:].tolist()
matches = [(match, similarity[match]) for match in matches]
matches.reverse()
matches

[(38, 0.5843799337008113),
 (36, 0.5724333016720691),
 (836, 0.5244194362041271),
 (40, 0.5126844935129918),
 (26, 0.5033481946111171)]

In [34]:
for m, match in enumerate(matches):
    print(f"Match {m+1} ({match[1]:.2f}) is chunk {vector_db[match[0]][0]}:\n{content_chunks[vector_db[match[0]][0]]}\n###################################################")

Match 1 (0.58) is chunk c38:
# 2.00-THE PLAYING FIELD

## 2.02 Home Base

Home base shall be marked by a five-sided slab of whitened rubber. It shall be a 17-inch square with two of the corners removed so that one edge is 17 inches long, two adjacent sides are 8\frac{1}{2} inches and the remaining two sides are 12 inches and set at an angle to make a point.
###################################################
Match 2 (0.57) is chunk c39:
# 2.00-THE PLAYING FIELD

## 2.02 Home Base

It shall be set in the ground with the point at the intersection of the lines extending from home base to first base and to third base; with the 17-inch edge facing the pitcher's plate, and the two 12-inch edges coinciding with the first and third base lines. The top edges of home base shall be beveled and the base shall be fixed in the ground level with the ground surface. (See drawing D in Appendix 2.) 3
###################################################
Match 3 (0.52) is chunk c838:
# APPENDICES

## Appen

### Generation: Q&A With Gemini Grounded With RAG

Provide the matched chunks of text along with the question as a prompt to a generative model for a grounded answer.

#### Prompt Building Function

Use the matching chunks as context for the prompt:

In [38]:
def get_prompt(question, top_n = 5):
    # get embedding for question
    question_embedding = embedder.get_embeddings([question])[0].values
    # get top_n matches:
    similarity = np.dot(question_embedding, vector_index.T)
    matches = np.argsort(similarity)[-top_n:].tolist()
    matches.reverse()
    matches = [[match, similarity[match]] for match in matches]
    # construct prompt:
    prompt = ''
    for m, match in enumerate(matches):
        prompt += f"Context {m+1}:\n{content_chunks[vector_db[match[0]][0]]}\n\n"
    prompt += f'Answer the following question using the provided contexts:\n{question}'
    
    return matches, prompt

In [39]:
matches, prompt = get_prompt(question) 
print(prompt)

Context 1:
# 2.00-THE PLAYING FIELD

## 2.02 Home Base

Home base shall be marked by a five-sided slab of whitened rubber. It shall be a 17-inch square with two of the corners removed so that one edge is 17 inches long, two adjacent sides are 8\frac{1}{2} inches and the remaining two sides are 12 inches and set at an angle to make a point.

Context 2:
# 2.00-THE PLAYING FIELD

## 2.02 Home Base

It shall be set in the ground with the point at the intersection of the lines extending from home base to first base and to third base; with the 17-inch edge facing the pitcher's plate, and the two 12-inch edges coinciding with the first and third base lines. The top edges of home base shall be beveled and the base shall be fixed in the ground level with the ground surface. (See drawing D in Appendix 2.) 3

Context 3:
# APPENDICES

## Appendix 2

Diagram No. 2 Layout at Home Plate, 1st, 2nd, and 3rd Bases 18" A 18" 90° LAYOUT AT SECOND BASE FOR LAYOUT AT PITCHER'S PLATE SEE DIAGRAM NO. 3 90° 6"

### Grounded Generation

In [40]:
answer = llm.generate_content(prompt).text
Markdown(answer)

The dimensions of a base depend on which base you are referring to:

* **Home Base:** It's a five-sided slab of whitened rubber with the following dimensions:
    * One edge: 17 inches
    * Two adjacent sides: 8 1/2 inches
    * Two remaining sides: 12 inches 
* **First, Second, and Third Bases:** These are marked by white canvas or rubber-covered bags with the following dimensions:
    * 18 inches square
    * 3 to 5 inches thick 


---
## Check Grounding API

Vertex AI Agent Builder has several helpful APIs for grounding, including:
- [Check Grounding API](https://cloud.google.com/generative-ai-app-builder/docs/check-grounding)
    - assess grounded-ness of responses
    
With this API you pass it the answer from an LLM along with chunks of context which all called facts in the API.  The response include an overall score, `support_score`, and a phrase by phrase breakdown of which chunk/fact is the citation for the phrase - if any.
    
**References:**

- [Discoveryengine Python Grounded Generation Service Client](https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.services.grounded_generation_service)

### Use Check Grounding API

Input LLM Answer and Context Chunks

In [41]:
ground_check = check_grounding_client.check_grounding(
    request = discoveryengine.CheckGroundingRequest(
        grounding_config = check_grounding_client.grounding_config_path(
            project = PROJECT_ID,
            location = 'global',
            grounding_config = "default_grounding_config",
        ),
        answer_candidate = answer,
        facts = [
            discoveryengine.GroundingFact(
                fact_text = content_chunks[vector_db[match[0]][0]],
                attributes = {'author': 'MLB', 'chunk_id': vector_db[match[0]][0]}
            )
            for match in matches
        ],
        grounding_spec = discoveryengine.CheckGroundingSpec(
            citation_threshold = 0.6,
            #enable_anti_citation = True,
            #anti_citation_threshold = 0.75
        ),
    )
)

In [42]:
ground_check

support_score: 0.854004442691803
cited_chunks {
  chunk_text: "# 2.00-THE PLAYING FIELD\n\n## 2.02 Home Base\n\nHome base shall be marked by a five-sided slab of whitened rubber. It shall be a 17-inch square with two of the corners removed so that one edge is 17 inches long, two adjacent sides are 8\\frac{1}{2} inches and the remaining two sides are 12 inches and set at an angle to make a point."
  source: "0"
}
cited_chunks {
  chunk_text: "# 2.00-THE PLAYING FIELD\n\n## 2.02 Home Base\n\nIt shall be set in the ground with the point at the intersection of the lines extending from home base to first base and to third base; with the 17-inch edge facing the pitcher\'s plate, and the two 12-inch edges coinciding with the first and third base lines. The top edges of home base shall be beveled and the base shall be fixed in the ground level with the ground surface. (See drawing D in Appendix 2.) 3"
  source: "1"
}
cited_chunks {
  chunk_text: "# Rule 2.03 to 2.05\n\n## 2.03 The Bases\n\nFir

### Examine Check Grounding Results Phrase-by-Phrase

In [43]:
overview = f"**Grounding Support Score (0-1):** {ground_check.support_score:.2f}\n|Answer Phrases|Citations|\n|---|---|\n"
all_ref_chunks = []
for claim in ground_check.claims:
    citations = [c for c in claim.citation_indices] if hasattr(claim, 'citation_indices') else []
    sources = [ground_check.cited_chunks[citation].source for citation in citations]
    ref_chunks = [vector_db[matches[int(source)][0]][0] for source in sources]
    if ref_chunks: all_ref_chunks += ref_chunks
    overview += f"|{claim.claim_text}|{ref_chunks}|\n"
overview += '-'*80 +'\n'    
for chunk_id in sorted(list(set(all_ref_chunks))):
    overview += f"**Chunk = {chunk_id}:**\n\n{content_chunks[chunk_id]}\n\n" + '-'*80 +'\n'

Markdown(overview.replace('#', ''))

**Grounding Support Score (0-1):** 0.85
|Answer Phrases|Citations|
|---|---|
|The dimensions of a base depend on which base you are referring to:|[]|
|* **Home Base:** It's a five-sided slab of whitened rubber with the following dimensions:|['c38']|
|* One edge: 17 inches|['c38', 'c39']|
|* Two adjacent sides: 8 1/2 inches|['c38']|
|* Two remaining sides: 12 inches|['c38']|
|* **First, Second, and Third Bases:** These are marked by white canvas or rubber-covered bags with the following dimensions:|['c40']|
|* 18 inches square|[]|
|* 3 to 5 inches thick|['c40']|
--------------------------------------------------------------------------------
**Chunk = c38:**

 2.00-THE PLAYING FIELD

 2.02 Home Base

Home base shall be marked by a five-sided slab of whitened rubber. It shall be a 17-inch square with two of the corners removed so that one edge is 17 inches long, two adjacent sides are 8\frac{1}{2} inches and the remaining two sides are 12 inches and set at an angle to make a point.

--------------------------------------------------------------------------------
**Chunk = c39:**

 2.00-THE PLAYING FIELD

 2.02 Home Base

It shall be set in the ground with the point at the intersection of the lines extending from home base to first base and to third base; with the 17-inch edge facing the pitcher's plate, and the two 12-inch edges coinciding with the first and third base lines. The top edges of home base shall be beveled and the base shall be fixed in the ground level with the ground surface. (See drawing D in Appendix 2.) 3

--------------------------------------------------------------------------------
**Chunk = c40:**

 Rule 2.03 to 2.05

 2.03 The Bases

First, second and third bases shall be marked by white canvas or rubber-covered bags, securely attached to the ground as indicated in Diagram 2. The first and third base bags shall be entirely within the infield. The second base bag shall be centered on second base. The bags shall be 18 inches square, not less than three nor more than five inches thick, and filled with soft material.

--------------------------------------------------------------------------------
